## Step:1 Load the workspace configurations

In [2]:
from azureml.core import Workspace, Dataset, Experiment

subscription_id = 'd0f7cb58-5c08-4ead-ac4c-29752d3ba0e1'
resource_group = 'mlops-aug-batch'
workspace_name = 'intellipat-mlops'

workspace = Workspace(subscription_id, resource_group, workspace_name)

## Step:2 Start with the experiment

In [4]:
experiment = Experiment(workspace=workspace, name="iris")
dataset = Dataset.get_by_name(workspace=workspace, name="Iris")

df = dataset.to_pandas_dataframe()
df.head(4)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa


## Step:3 Create a DT classifier

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import joblib

X = df[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
Y = df['Species']

run = experiment.start_logging()

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size= 0.40, shuffle=True)


#step1: initialise the model class
clf = DecisionTreeClassifier(criterion="gini", max_depth=4)
#step:2 train the model with training data
clf.fit(X_train, y_train)

#step-3 evaluate the model with testing data
y_pred = clf.predict(X_test)
accurcay = accuracy_score(y_test,y_pred)

run.log("accuracy",accurcay*100 )

model_name =  "iris_decision_tree.pkl"
filepath = "outputs/"+model_name

joblib.dump(value= clf,filename=filepath)
run.upload_file(name=model_name, path_or_stream=filepath)
run.complete()

### Step:4 Register the model

In [22]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
import sklearn

In [26]:
model = Model.register(
    workspace=workspace,
    model_name = "iris",
    model_path = filepath,
    model_framework= Model.Framework.SCIKITLEARN,
    model_framework_version= sklearn.__version__,
    description= "This is a sample DT classifier",
    resource_configuration= ResourceConfiguration(cpu=1, memory_in_gb= 0.5)
)

print("Name:", model.name)
print("version:", model.version)

Registering model iris
Name: iris
version: 4
